In [1]:
import time
import random
import numpy as np
import torch
import json

import messages
from simulator import Simulator
from agent import Agent
from unity_env import UnityEnv

<class 'sac.sac_agent.SACAgent'>
<class 'sac.sac_env.SACEnv'>


In [2]:
simulator = Simulator(port=5004)

In [3]:
env = UnityEnv(simulator, 9, 2, 1, lambda: 2*np.random.random((2,))-1, "SimpleUnityEnv")
agent = Agent(env)

In [ ]:
cp = torch.load(f"save/{env.get_name()}/checkpoint.pt")
agent.policy.load_state_dict(cp['pi'])
agent.q1.load_state_dict(cp['q1'])
agent.q2.load_state_dict(cp['q2'])
agent.replay_buffer = cp["replay_buffer"]
from_epoch = cp['epoch']

In [4]:
simulator.start_server()
serv = simulator.server

In [6]:
env.set_time_scale(1)

In [ ]:
agent.train(from_epoch=0, epoch_len=2000, start_steps=4000)

In [ ]:
msg = messages.RequestMessage("test", "").to_json()
msg2 = messages.RequestMessage("timeScale", messages.SingleFieldMessage(2.).to_json()).to_json()
msg3 = messages.RequestMessage("pause", messages.SingleFieldMessage(False).to_json()).to_json()
msg4 = messages.RequestMessage("reset", "").to_json()
msg5 = messages.RequestMessage("step", messages.ControllMessage(5, 5).to_json()).to_json()


In [ ]:
simulator.server.clients[-1].socket.sendall(msg4.encode())

In [ ]:
serv.clients[-1].socket.sendall(msg5.encode())
time.sleep(0.02)

In [15]:
r = lambda d: {k:round(v, 6) for (k, v) in d.items()}

print(r(simulator._reset().velocity))
time.sleep(env.get_target_frame_duration())
print(r(simulator._step(10, 0).velocity))
for _ in range(100):
    time.sleep(env.get_target_frame_duration())
    print(r(simulator._step(0, 0).velocity))

{'x': 0.0, 'y': 0.0, 'z': 0.0}
{'x': 1e-06, 'y': 0.0, 'z': -0.0}
{'x': -2.074452, 'y': -0.002044, 'z': -7.24097}
{'x': -1.94594, 'y': -0.001406, 'z': -6.787168}
{'x': -1.882703, 'y': -0.000866, 'z': -6.559699}
{'x': -1.757893, 'y': -0.002393, 'z': -6.107535}
{'x': -1.695966, 'y': -0.010908, 'z': -5.890994}
{'x': -1.563354, 'y': -0.001278, 'z': -5.427142}
{'x': -1.305213, 'y': -0.000813, 'z': -4.520866}
{'x': -1.241622, 'y': -0.001358, 'z': -4.294851}
{'x': -1.113838, 'y': -0.00142, 'z': -3.841708}
{'x': -1.049804, 'y': -0.001406, 'z': -3.61512}
{'x': -0.857034, 'y': -0.001226, 'z': -2.935373}
{'x': -0.790266, 'y': -0.001637, 'z': -2.710112}
{'x': -0.660603, 'y': -0.000184, 'z': -2.255612}
{'x': -0.596208, 'y': -0.000704, 'z': -2.029793}
{'x': -0.46643, 'y': -0.003423, 'z': -1.580539}
{'x': -0.33495, 'y': -0.003181, 'z': -1.12808}
{'x': -0.202654, 'y': -0.002823, 'z': -0.675714}
{'x': -0.069414, 'y': -0.002477, 'z': -0.223636}
{'x': -0.008594, 'y': -0.002173, 'z': -0.024776}
{'x': 0.002

In [ ]:

response = messages.ResponseMessage.from_json(simulator.server.clients[-1].receive().decode())
observation_message = messages.ObservationMessage.from_json(response.value)
print(json.dumps(observation_message.__dict__, indent=4))

In [ ]:

response = messages.ResponseMessage.from_json(simulator.server.clients[-1].receive().decode())
observation_message = messages.ObservationMessage.from_json(response.value)
print(json.dumps(observation_message.__dict__, indent=4))

In [26]:
simulator.shutdown()
serv.stop_server()

In [ ]:
serv.server_socket.close()

In [ ]:
simulator.server.clients = []

In [ ]:
simulator.shutdown()

In [ ]:
2*np.random.random((2,))-1

In [ ]:
observation_message.redBallPosition,observation_message.agentPostion

In [ ]:
simulator.server.clients[-1].receive()[0]